In [ ]:
#
#
# spit and embed by chunking
#
#

import re
import uuid
import chromadb
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings

persistent_client = chromadb.PersistentClient()

collection_name = 'all-MiniLM-L6-v2_1000_split_clean'

try:
    persistent_client.delete_collection(name=collection_name)
except:
    print('nothing to delete')

collection = persistent_client.get_or_create_collection(collection_name)

embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

pdfs = ['London_Borough_of_Southwark',
        'London_Borough_of_Tower_Hamlets',
        'London_Borough_of_Islington']

for pdf in pdfs[0:3]:
    with open('txts/' + pdf + '.txt') as f:
        string = f.read()
        cleaner_string = string.replace('\n', ' ').replace('\r', '')

        clean_string = re.sub("\s\s+", " ", cleaner_string)

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=100,
            length_function=len,
            is_separator_regex=False,
        )

        # Split text
        split_texts = text_splitter.create_documents([clean_string])
        split_texts_list = [str(txt.page_content) for txt in split_texts]
        display(len(split_texts_list))

        # Embed text
        embedded_texts = embedding_model.embed_documents(
            texts=split_texts_list)

        # add vectors to collection
        ids = [str(uuid.uuid4()) for sent in split_texts_list]
        metadatas = [{"LPA": pdf}
                     for sent in split_texts_list]
        collection.add(
            embeddings=embedded_texts,
            documents=split_texts_list,
            ids=ids,
            metadatas=metadatas
        )

In [1]:
#
#
# split and embed by sentences
#
#

import chromadb
from nltk.tokenize import sent_tokenize
from langchain.embeddings import SentenceTransformerEmbeddings
import re
from helpers import embed, pinecone_embed

collection_name = 'all-MiniLM-L6-v2_sentence_split'

persistent_client = chromadb.PersistentClient()

try:
    persistent_client.delete_collection(name=collection_name)
except:
    print('nothing to delete')


collection = persistent_client.get_or_create_collection(collection_name)
print('Collection deleted?: ')
display(collection.count())
embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

pdfs = ['London_Borough_of_Southwark',
        'London_Borough_of_Tower_Hamlets', 'London_Borough_of_Islington']


for pdf in pdfs[0:1]:
    with open('txts/' + pdf + '.txt') as f:
        string = f.read()
        cleaner_string = string.replace('\n', ' ').replace('\r', '')

        clean_string = re.sub("\s\s+", " ", cleaner_string)

        # Split text
        split_texts_list = sent_tokenize(clean_string)

        proper_setences = [
            i for i in split_texts_list if i.count(' ') >= 6]

        print('number of sentences to embed')
        print(len(proper_setences))

        n = 3  # group size
        m = 2  # overlap size
        triplets = [" ".join(proper_setences[i:i+n])
                    for i in range(0, len(proper_setences), n-m)]

        triplets_trucated = [
            i[:1000] for i in triplets]

        embed(collection, embedding_model, proper_setences, pdf)

Collection deleted?: 


0

/Users/jimmy/Projects/localplans/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


number of sentences to embed
3352
3352
3352
3352


ValueError: Expected ID to be a str, got ('ea917922-6bf9-421b-b2b0-df973ba4d0ef', [0.007049312349408865, 0.0037307392340153456, 0.05868951231241226, -0.12151091545820236, -0.026203883811831474, 0.06338552385568619, -0.06024651601910591, -0.018123134970664978, -0.15988048911094666, 0.0583149716258049, -0.0285935141146183, -0.060437120497226715, 0.021241866052150726, 0.027746431529521942, -0.0008897087536752224, 0.0534585602581501, 0.031648579984903336, -0.07946105301380157, -0.028515184298157692, -0.015992064028978348, 0.07133373618125916, -0.023155035451054573, 0.018104810267686844, -0.0049970513209700584, -0.09808901697397232, -0.01911879889667034, 0.07275884598493576, 0.014110165648162365, -0.04004855453968048, 0.06201932579278946, 0.08114098012447357, 0.05467699468135834, -0.05132041126489639, -0.04241568222641945, 0.04462660849094391, 0.004436573479324579, 0.02685990184545517, -0.03510517254471779, 0.05156895890831947, -0.06374090909957886, -0.030079089105129242, -0.130620241165161

In [4]:
#
#
# split and embed by sentences TO PINECONE
#
#


from nltk.tokenize import sent_tokenize
from langchain.embeddings import SentenceTransformerEmbeddings
import re
import os
from helpers import pinecone_embed, pinecone_connect

index = pinecone_connect()

index.delete(delete_all=True, namespace='localplans')

embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

pdfs = os.listdir('txts')
pdfs = sorted(pdfs)

for pdf in pdfs:
    print('txts/' + pdf)
    with open('txts/' + pdf) as f:
        string = f.read()
        cleaner_string = string.replace('\n', ' ').replace('\r', '')

        clean_string = re.sub("\s\s+", " ", cleaner_string)

        # Split text
        split_texts_list = sent_tokenize(clean_string)

        proper_sentences = [
            i for i in split_texts_list if i.count(' ') >= 6]

        print('number of sentences to embed')
        print(len(proper_sentences))

        n = 3  # group size
        m = 2  # overlap size
        triplets = [" ".join(proper_sentences[i:i+n])
                    for i in range(0, len(proper_sentences), n-m)]

        triplets_trucated = [
            i[:1000] for i in triplets]

    n = 500
    for i in range(0, len(triplets_trucated), n):
        chunk = triplets_trucated[i:i + n]
        pinecone_embed(index, embedding_model, chunk, pdf)

txts/Allerdale_Borough_Council.txt
number of sentences to embed
1880
500
500
500
500
500
500
500
500
500
380
380
380
txts/Arun_District_Council.txt
number of sentences to embed
895
500
500
500
395
395
395
txts/Ashfield_District_Council.txt
number of sentences to embed
3624
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
124
124
124
txts/Barnsley_Metropolitan_Borough_Council.txt
number of sentences to embed
2795
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
295
295
295
txts/Basildon_Borough_Council.txt
number of sentences to embed
4929
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
429
429
429
txts/Bassetlaw_District_Council.txt
number of sentences to embed
1509
500
500
500
500
500
500
500
500
500
9
9
9
txts/Bath_and_North_East_Somerset_Council.txt
number of sentences to embed
1704
500
500
500
500
500
500
500
500
500
204
204
204
txts/Bedford_Borough_Council.txt
number of senten

In [2]:
from helpers import pinecone_connect
index = pinecone_connect()

/Users/jimmy/Projects/localplans/venv/lib/python3.11/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
